#### Module Initialization

In [1]:
## -- Module WAXS Classes -- ##
from WAXSDiffSim import WAXSDiffSim
from MoleculeConstructor import MoleculeConstructor

# - Import Relevant Modules"
import xarray as xr
import numpy as np
import pathlib, os
from pathlib import Path
from typing import Optional
import matplotlib.pyplot as plt
from math import cos, sin, radians, degrees
import pandas as pd

# - pymatgen: https://pymatgen.org/
# from pymatgen import Structure, Lattice
from pymatgen.core import Structure, Lattice, Molecule, IMolecule
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer
from pymatgen.io.cif import CifWriter

#### Path Definitions

In [14]:
# -- Base Path Definitions -- #
basePath = pathlib.Path('/Users/keithwhite/github_repositories/pyWAXS/examples_local/simulations')

# -- Project Path Definitions -- #
cifPath = basePath.joinpath('cif')
waxsPath = basePath.joinpath('waxs')
moleculePath = basePath.joinpath('molecules')


#### Example: Single-Atom Basis

In [25]:
# from pymatgen.core import Structure, Lattice
# from pymatgen.symmetry.analyzer import SpacegroupAnalyzer

# Create a cubic lattice
lattice = Lattice.cubic(3.567)

# Specify species and coordinates
species = ['C']
coords = [[0, 0, 0]]  # Wyckoff position 'a' for Fd-3m

# Create a Structure object
structure = Structure.from_spacegroup("Fd-3m", lattice, species, coords)

# Analyze the structure with the spacegroup analyzer
analyzer = SpacegroupAnalyzer(structure)

# Get the primitive standardized structure
prim_structure = analyzer.get_primitive_standard_structure()

# Print information
print("Spacegroup Symbol:", analyzer.get_space_group_symbol())
print("International Number:", analyzer.get_space_group_number())
print("Crystal System:", analyzer.get_crystal_system())

# Write CIF file to cifPath
cif_file_path = cifPath.joinpath("diamond.cif")
writer = CifWriter(prim_structure)
writer.write_file(cif_file_path)


Spacegroup Symbol: Fd-3m
International Number: 227
Crystal System: cubic


#### Example: Multi-Atom Basis

In [26]:
# Create a cubic lattice
lattice = Lattice.cubic(5.692)

# Specify species and coordinates
species = ['Na', 'Cl']
coords = [[0, 0, 0], [0.5, 0.5, 0.5]]  # Wyckoff positions 'a' and 'b' for Fm-3m

# Create a Structure object
structure = Structure.from_spacegroup("Fm-3m", lattice, species, coords)

# Analyze the structure with the spacegroup analyzer
analyzer = SpacegroupAnalyzer(structure)

# Get the primitive standardized structure
prim_structure = analyzer.get_primitive_standard_structure()

# Print information
print("Spacegroup Symbol:", analyzer.get_space_group_symbol())
print("International Number:", analyzer.get_space_group_number())
print("Crystal System:", analyzer.get_crystal_system())

# Write CIF file to cifPath
cif_file_path = cifPath.joinpath("NaCl.cif")
writer = CifWriter(prim_structure)
writer.write_file(cif_file_path)

Spacegroup Symbol: Fm-3m
International Number: 225
Crystal System: cubic


#### Example: Generate CIF with Fractional Occupancy

In [5]:
# Define lattice and species
lattice = Lattice.cubic(5.692)
species = [{"Na": 0.5, "K": 0.5}, {"Cl": 1.0}]
coords = [[0, 0, 0], [0.5, 0.5, 0.5]]

# Create the Structure object
structure = Structure(lattice, species, coords)

# Write to CIF file
cif_file_path = cifPath.joinpath("Na_K_Cl_mixed.cif")
writer = CifWriter(structure)
writer.write_file(cif_file_path)

#### PbI3Cl Structure

In [28]:
from pymatgen.core import Lattice, Structure
from pymatgen.io.cif import CifWriter
from pathlib import Path
import numpy as np

# Lattice constants
a, b, c = 17.816, 26.762, 5.727
d1, d2 = 2.89, 3.11 # Pb - Cl, Pb - I bond distances

# Create an orthorhombic lattice
lattice = Lattice.orthorhombic(a, b, c)

# -- Pb Coordinate Positions
# Pb atom positions in the unit cell
pb_coords_cartesian = [
    [a/4, 0, 0], [3*a/4, 0, 0], [0, a/4, 0], [0, b - a/4, 0],
    [a/4, b, 0], [3*a/4, b, 0], [a, a/4, 0], [a, b - a/4, 0],
    [a/4, 0, c], [3*a/4, 0, c], [0, a/4, c], [0, b - a/4, c],
    [a/4, b, c], [3*a/4, b, c], [a, a/4, c], [a, b - a/4, c],
    [a/4, b/2, 0], [3*a/4, b/2, 0], [a/2, a/2, 0], [a/2, b - a/2, 0],
    [a/4, b/2, c], [3*a/4, b/2, c], [a/2, a/2, c], [a/2, b - a/2, c]
]

# Convert to fractional coordinates and round to the third decimal place
pb_coords_fractional = [[round(x/a, 3), round(y/b, 3), round(z/c, 3)] for x, y, z in pb_coords_cartesian]
# pb_coords_fractional[:5]  # Display the first 5 for a quick look

# -- Chloride Coordinate Positions
cl_coords_cartesian = [
    [a/4, 0, d1], [3*a/4, 0, d1], [0, a/4, d1], [0, b - a/4, d1],
    [a/4, b, d1], [3*a/4, b, d1], [a, a/4, d1], [a, b - a/4, d1],
    [a/4, b/2, d1], [3*a/4, b/2, d1], [a/2, a/2, d1], [a/2, b - a/2, d1]
]

cl_coords_fractional = [[round(x/a, 3), round(y/b, 3), round(z/c, 3)] for x, y, z in cl_coords_cartesian]

# -- Iodide Coordinate Positions
# Function to rotate a point in a plane
def rotate_point(x, y, angle_deg):
    angle_rad = np.radians(angle_deg)
    x_new = x * np.cos(angle_rad) - y * np.sin(angle_rad)
    y_new = x * np.sin(angle_rad) + y * np.cos(angle_rad)
    return x_new, y_new

# Initialize list to store iodide coordinates
iodide_coords = []

# Generate iodide coordinates around each Pb atom
for x, y, z in pb_coords_cartesian:
    for angle in [45, 135, 225, 315]:  # Four iodide atoms at these angles
        dx, dy = rotate_point(d2, 0, angle)
        new_coord = [x + dx, y + dy, z]
        iodide_coords.append(new_coord)

# Convert to NumPy array for easier manipulation
iodide_coords = np.array(iodide_coords)

# Remove duplicates and round to 3 decimal places
unique_i_coords = np.unique(np.round(iodide_coords, 3), axis=0)

# Apply constraints to remove coordinates outside the unit cell
constrained_i_coords = [
    coord for coord in unique_i_coords.tolist()
    if all(0 <= x <= dim for x, dim in zip(coord, [a, b, c]))
]

iodide_coords_fractional = [[round(x/a, 3), round(y/b, 3), round(z/c, 3)] for x, y, z in constrained_i_coords]
# iodide_coords_fractional[:5]  # Number of constrained iodide atoms and first 5 coordinates

# -- Potassium (methylammonium (MA) electron equilvalent)
k_coords = [
    [0, 0, c/2], [a/2, 0, c/2], [a, 0, c/2], 
    [0, b, c/2], [a/2, b, c/2], [a, b, c/2],  # width - perimeter
    [0, a/2, c/2], [0, b/2, c/2], [0, b - a/2, c/2], 
    [a, a/2, c/2], [a, b/2, c/2], [a, b - a/2, c/2],  # length - perimeter
    [a/2, b/2, c/2],  # center
    [a/2, a/4, c/2], [a/2, b - a/4, c/2],  # center - sides
    [a/4, a/4, c/2], [a/4, a/2, c/2], [a/4, b - a/4, c/2], [a/4, b - a/2, c/2],
    [3*a/4, a/4, c/2], [3*a/4, a/2, c/2], [3*a/4, b - a/4, c/2], [3*a/4, b - a/2, c/2]
]

k_coords_fractional = [[round(x/a, 3), round(y/b, 3), round(z/c, 3)] for x, y, z in k_coords]

# -- Generate the CIF
coords = pb_coords_fractional + cl_coords_fractional + iodide_coords_fractional + k_coords_fractional

# Specify species for Pb atoms
species = ["Pb2+"] * 24 + ['Cl-'] * 12 + ['I-'] * len(iodide_coords_fractional) + ['K'] * len(k_coords_fractional)

# Create the Structure object
structure = Structure(lattice, species, coords)

# Define the path to save the CIF file
cif_file_path = cifPath / "MA2Pb3Cl_K_recreated.cif"

# Write to CIF file
writer = CifWriter(structure)
writer.write_file(cif_file_path)

#### (MA)2PbI4 + MAI

In [ ]:
from pymatgen.core import Lattice, Structure
from pymatgen.io.cif import CifWriter
from pathlib import Path
import numpy as np

# Lattice constants
a, b, c = 17.816, 26.762, 6.24
d1, d2 = 3.12, 3.12 # Pb - Cl, Pb - I bond distances

# Create an orthorhombic lattice
lattice = Lattice.orthorhombic(a, b, c)

# -- Pb Coordinate Positions
# Pb atom positions in the unit cell
pb_coords_cartesian = [
    [a/4, 0, 0], [3*a/4, 0, 0], [0, a/4, 0], [0, b - a/4, 0],
    [a/4, b, 0], [3*a/4, b, 0], [a, a/4, 0], [a, b - a/4, 0],
    [a/4, 0, c], [3*a/4, 0, c], [0, a/4, c], [0, b - a/4, c],
    [a/4, b, c], [3*a/4, b, c], [a, a/4, c], [a, b - a/4, c],
    [a/4, b/2, 0], [3*a/4, b/2, 0], [a/2, a/2, 0], [a/2, b - a/2, 0],
    [a/4, b/2, c], [3*a/4, b/2, c], [a/2, a/2, c], [a/2, b - a/2, c]
]

# Convert to fractional coordinates and round to the third decimal place
pb_coords_fractional = [[round(x/a, 3), round(y/b, 3), round(z/c, 3)] for x, y, z in pb_coords_cartesian]
# pb_coords_fractional[:5]  # Display the first 5 for a quick look

# -- Chloride Coordinate Positions
i_d1_coords_cartesian = [
    [a/4, 0, d1], [3*a/4, 0, d1], [0, a/4, d1], [0, b - a/4, d1],
    [a/4, b, d1], [3*a/4, b, d1], [a, a/4, d1], [a, b - a/4, d1],
    [a/4, b/2, d1], [3*a/4, b/2, d1], [a/2, a/2, d1], [a/2, b - a/2, d1]
]

i_d1_coords_fractional = [[round(x/a, 3), round(y/b, 3), round(z/c, 3)] for x, y, z in cl_coords_cartesian]

# -- Iodide Coordinate Positions
# Function to rotate a point in a plane
def rotate_point(x, y, angle_deg):
    angle_rad = np.radians(angle_deg)
    x_new = x * np.cos(angle_rad) - y * np.sin(angle_rad)
    y_new = x * np.sin(angle_rad) + y * np.cos(angle_rad)
    return x_new, y_new

# Initialize list to store iodide coordinates
iodide_coords = []

# Generate iodide coordinates around each Pb atom
for x, y, z in pb_coords_cartesian:
    for angle in [45, 135, 225, 315]:  # Four iodide atoms at these angles
        dx, dy = rotate_point(d2, 0, angle)
        new_coord = [x + dx, y + dy, z]
        iodide_coords.append(new_coord)

# Convert to NumPy array for easier manipulation
iodide_coords = np.array(iodide_coords)

# Remove duplicates and round to 3 decimal places
unique_i_coords = np.unique(np.round(iodide_coords, 3), axis=0)

# Apply constraints to remove coordinates outside the unit cell
constrained_i_coords = [
    coord for coord in unique_i_coords.tolist()
    if all(0 <= x <= dim for x, dim in zip(coord, [a, b, c]))
]

iodide_coords_fractional = [[round(x/a, 3), round(y/b, 3), round(z/c, 3)] for x, y, z in constrained_i_coords]
# iodide_coords_fractional[:5]  # Number of constrained iodide atoms and first 5 coordinates

# -- Potassium (methylammonium (MA) electron equilvalent)
k_coords = [
    [0, 0, c/2], [a/2, 0, c/2], [a, 0, c/2], 
    [0, b, c/2], [a/2, b, c/2], [a, b, c/2],  # width - perimeter
    [0, a/2, c/2], [0, b/2, c/2], [0, b - a/2, c/2], 
    [a, a/2, c/2], [a, b/2, c/2], [a, b - a/2, c/2],  # length - perimeter
    [a/2, b/2, c/2],  # center
    [a/2, a/4, c/2], [a/2, b - a/4, c/2],  # center - sides
    [a/4, a/4, c/2], [a/4, a/2, c/2], [a/4, b - a/4, c/2], [a/4, b - a/2, c/2],
    [3*a/4, a/4, c/2], [3*a/4, a/2, c/2], [3*a/4, b - a/4, c/2], [3*a/4, b - a/2, c/2]
]

k_coords_fractional = [[round(x/a, 3), round(y/b, 3), round(z/c, 3)] for x, y, z in k_coords]

# -- Generate the CIF
coords = pb_coords_fractional + i_d1_coords_fractional + iodide_coords_fractional + k_coords_fractional

# Specify species for Pb atoms
species = ["Pb2+"] * 24 + ['I-'] * 12 + ['I-'] * len(iodide_coords_fractional) + ['K'] * len(k_coords_fractional)

# Create the Structure object
structure = Structure(lattice, species, coords)

# Define the path to save the CIF file
cif_file_path = cifPath / "MA2Pb4_K_recreated.cif"

# Write to CIF file
writer = CifWriter(structure)
writer.write_file(cif_file_path)

#### Example: Molecule Constructor Class

In [5]:
# Test the MoleculeConstructor class
filepath = Path('/Users/keithwhite/github_repositories/pyWAXS/examples_local/simulations/molecules/example_dmf_bonddistances.xlsx')

# Initialize the MoleculeConstructor with the Excel filepath
constructor = MoleculeConstructor(filepath=filepath)

constructor.df_bond_distances
# # Set the origin atom (Oxygen in this case)
# constructor.set_origin(origin_atom = 'O', element= 'O')

# # Start constructing the molecule
# constructor.construct_molecule(current_atom = 'O', element = 'O')

# # Save to CIF file
# constructor.save_to_cif(cif_filepath=cifPath, lattice_parameter=20.0)

# # View the resulting DataFrame
# print(constructor.atom_df)

distance_angstr_experimental  distance_angstr_ideal
atom_i atom_j                                                     
C1     N                              1.491                  1.465
       H11                            1.116                  1.090
       H12                            1.114                  1.090
       H13                            1.115                  1.089
C2     N                              1.485                  1.464
       H21                            1.115                  1.089
       H22                            1.115                  1.090
       H23                            1.114                  1.090
C      O                              1.226                  1.213
       N                              1.326                  1.347
       HC                             1.103                  1.080

#### Molecule Definitions

In [27]:
def water_molecule():
    angle = 104.5  # H-O-H angle in degrees
    angle_rad = radians(angle)
    bond_length = 0.9572  # O-H bond length in Angstrom

    # Coordinates with O at the center of the unit cell in fractional coordinates
    O = [0.5, 0.5, 0.5]
    H1 = [0.5 + bond_length * cos(angle_rad / 2) / 20, 
          0.5 + bond_length * sin(angle_rad / 2) / 20, 
          0.5]
    H2 = [0.5 - bond_length * cos(angle_rad / 2) / 20, 
          0.5 + bond_length * sin(angle_rad / 2) / 20, 
          0.5]
    
    return Molecule(["O", "H", "H"], [O, H1, H2])

def dimethylformamide():
    coords = [
        # O, C, N
        [0.000, 0.000, 0.000],

        [0.000, 0.000, 1.200],
        [0.000, 0.000, 2.400],
        # H atoms on N
        [0.866, 0.000, 2.900],
        [-0.866, 0.000, 2.900],
        # H atoms on C (methyl)
        [0.866, 0.000, -0.500],
        [-0.433, 0.750, -0.500],
        [-0.433, -0.750, -0.500]
    ]
    return Molecule(["O", "C", "N", "H", "H", "H", "H", "H"], coords)

def methylammonium():
    coords = [
        [0.000, 0.000, 0.000],  # N
        [0.000, 0.000, 1.000],  # H
        [0.866, 0.000, -0.500],  # H
        [-0.433, 0.750, -0.500],  # H
        [-0.433, -0.750, -0.500],  # H
        [0.000, 0.000, -1.500],  # C
        [0.000, 0.000, -2.500]  # H (methyl)
    ]
    return Molecule(["N", "H", "H", "H", "H", "C", "H"], coords)

def dimethylsulfoxide():
    coords = [
        # S, O
        [0.000, 0.000, 0.000],
        [0.000, 0.000, 1.600],
        # C atoms
        [0.866, 0.000, -0.500],
        [-0.866, 0.000, -0.500],
        # H atoms on C (methyl)
        [1.732, 0.000, 0.000],
        [0.433, 0.750, -1.000],
        [0.433, -0.750, -1.000],
        [-1.732, 0.000, 0.000],
        [-0.433, 0.750, -1.000],
        [-0.433, -0.750, -1.000]
    ]
    return Molecule(["S", "O", "C", "C", "H", "H", "H", "H", "H", "H"], coords)

def formamidinium():
    coords = [
        [0.000, 0.000, 0.000],  # C
        [0.866, 0.000, 0.500],  # H
        [-0.433, 0.750, 0.500],  # H
        [-0.433, -0.750, 0.500],  # H
        [0.000, 0.000, 1.500],  # N
        [0.866, 0.000, 2.000],  # H
        [-0.866, 0.000, 2.000],  # H
    ]
    return Molecule(["C", "H", "H", "H", "N", "H", "H"], coords)


#### Generate CIFs of Molecules

In [28]:
# Create a large cubic lattice
lattice = Lattice.cubic(20.0)  # 20 Angstroms on a side

# Use the corrected water molecule
water = water_molecule()
species = water.species
coords = water.cart_coords

# Create Structure
structure = Structure(lattice, species, coords)

# Save to CIF
cif_file_path = cifPath.joinpath("corrected_centered_water.cif")
writer = CifWriter(structure)
writer.write_file(cif_file_path)

#### Example: Assigning Charge to Atomic Species

In [12]:
# Cubic lattice
lattice = Lattice.cubic(5.64)  # Lattice parameter for NaCl

# Species and Coordinates with charges
species = ["Na1+", "Cl1-"]
coords = [[0, 0, 0], [0.5, 0.5, 0.5]]

# Create Structure
structure = Structure(lattice, species, coords)

# Save to CIF
cif_file_path = cifPath.joinpath("NaCl_charged.cif")
writer = CifWriter(structure)
writer.write_file(cif_file_path)